🛠️ 1. 기본 준비

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# 데이터를 학습용과 테스트용으로 분리
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# 평균 제곱 오차 계산
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
%matplotlib

# 데이터 불러오기
train = pd.read_csv('data3/train.csv')
test = pd.read_csv('data3/test.csv')


Using matplotlib backend: module://matplotlib_inline.backend_inline


🧱 2. 범주형 변수 변환 (factor → category)

In [42]:
cat_vars = ['season', 'holiday', 'workingday', 'weather']
for col in cat_vars:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')


⏱️ 3. 시간(datetime) 처리

In [43]:
# datetime 컬럼을 datetime 객체로 변환
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

# 시간만 추출
train['time'] = train['datetime'].dt.strftime('%H:%M:%S')
test['time'] = test['datetime'].dt.strftime('%H:%M:%S')


📆 4. 요일(day) 변수 생성

In [44]:
train['day'] = train['datetime'].dt.day_name()
test['day'] = test['datetime'].dt.day_name()

train['day'] = train['day'].astype('category')
test['day'] = test['day'].astype('category')

🌞 5. 일요일 여부 변수 생성

In [45]:
train['sunday'] = (train['day'] == 'Sunday').astype('category')
test['sunday'] = (test['day'] == 'Sunday').astype('category')


⏰ 6. 시간(hour) 및 시간대(daypart) 파생

In [46]:
# 시간 정수 추출
train['hour'] = train['datetime'].dt.hour
test['hour'] = test['datetime'].dt.hour

# 시간대(daypart) 초기화
def get_daypart(hour):
    if 4 < hour < 10:
        return 1
    elif 10 <= hour < 16:
        return 2
    elif 15 < hour < 22:
        return 3
    else:
        return 4

train['daypart'] = train['hour'].apply(get_daypart).astype('category')
test['daypart'] = test['hour'].apply(get_daypart).astype('category')

# hour도 범주형 처리
train['hour'] = train['hour'].astype('category')
test['hour'] = test['hour'].astype('category')


✅ 최종 전처리된 변수 예시

In [47]:
train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,time,day,sunday,hour,daypart
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,00:00:00,Saturday,False,0,4
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,01:00:00,Saturday,False,1,4
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,02:00:00,Saturday,False,2,4
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,03:00:00,Saturday,False,3,4
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,04:00:00,Saturday,False,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19:00:00,Wednesday,False,19,3
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20:00:00,Wednesday,False,20,3
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21:00:00,Wednesday,False,21,3
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,22:00:00,Wednesday,False,22,4


In [48]:
xtr = train[['datetime', 'season', 'holiday', 'workingday', 'weather',
       'temp', 'atemp', 'humidity', 'windspeed',
       'time', 'day', 'sunday', 'hour', 'daypart']]

ytr = train['count']


In [49]:
xtt = test[['datetime', 'season', 'holiday', 'workingday', 'weather',
       'temp', 'atemp', 'humidity', 'windspeed',
       'time', 'day', 'sunday', 'hour', 'daypart']]

🎁 추가 팁

In [50]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# 1. 데이터 불러오기
train['datetime'] = pd.to_datetime(train['datetime'])

# 3. 피처 리스트
num_features = ['temp', 'atemp', 'humidity', 'windspeed']
cat_features = ['season', 'holiday', 'workingday', 'weather', 'hour', 'daypart', 'sunday']

# 4. X, y 나누기
X = train[num_features + cat_features]
y = train['count']

# 5. 훈련/검증 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=156)

# 6. 파이프라인 구성
preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

model = Pipeline([
    ('preprocess', preprocessor),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 7. 모델 학습
model.fit(X_train, y_train)

# 8. 예측 및 평가
y_pred = model.predict(X_valid)
rmsle = np.sqrt(mean_squared_log_error(y_valid, y_pred))
print(f'검증 데이터 RMSLE: {rmsle:.4f}')


검증 데이터 RMSLE: 0.4455


In [53]:
md = model.predict(test)

In [54]:
submission = pd.DataFrame({
    'datetime' : test['datetime'],
    'count'    : md
})

submission['count'] = submission['count'].round().astype(int)

submission.to_csv('sub.csv',index=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

print(f"RMSLE 평균: {-scores.mean():.4f}")

RMSLE 평균: 112.5148
